In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "tiiuae/falcon-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    load_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = True

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from datasets import load_dataset

dataset_name = "nebulae7/eight-db"
dataset = load_dataset(dataset_name, split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/nebulae7___json/nebulae7--eight-db-3533d36754bbda7a/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


In [ ]:
dataset[10]

{'question': "What impact did the recent update from Canaccord have on Fidelity International's adjusted earnings per share for 2023 and 2024?",
 'answer': "As a result, Totally issued guidance for roughly 6.3m in underlying earnings in 2023, while net cash was expected to be 5.5m on 31 March, and Canaccord updated its estimates to reflect the firm's update, which results in a reduction of 31% and 22% to adjusted earnings per share for 2023 and 2024."}

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import prepare_model_for_kbit_training
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model) #basically adapter or wrapper around the model

In [ ]:
from peft import (
    LoraConfig,
    PeftConfig,
    get_peft_model
)

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["query_key_value"],
)
model = get_peft_model(model, peft_config)
print_trainable_parameters(model)

trainable params: 18874368 || all params: 3627619200 || trainable%: 0.5202962868870029


In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 100
generation_config.temperature = 0.5
generation_config.top_p = 0.8
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 11,
  "max_new_tokens": 100,
  "pad_token_id": 11,
  "temperature": 0.5,
  "top_p": 0.8,
  "transformers_version": "4.31.0"
}

In [ ]:
#Inference before training
prompt = f"""
<human>: Can you explain the range of investment options available to customers?
<assistance>:
""".strip()
print(prompt)

<human>: Can you explain the range of investment options available to customers?
<assistance>:


In [ ]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["question"]}
<assistance>: {data_point["answer"]}
  """.strip()
def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt,padding=True, truncation=True)
  return tokenized_full_prompt
dataset = dataset.shuffle().map(generate_and_tokenize_prompt)
print(dataset.shape)

Map:   0%|          | 0/101043 [00:00<?, ? examples/s]

(101043, 5)


In [ ]:
OUTPUT_DIR = "experiments"

In [ ]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs

<IPython.core.display.Javascript object>

In [ ]:
import transformers

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=5,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=4,
    logging_steps=10,
    output_dir=OUTPUT_DIR,
    max_steps=1000,
    optim="paged_adamw_8bit",
    lr_scheduler_type = 'cosine',
    warmup_ratio = 0.01,
    report_to = 'tensorboard'
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.933500
20,2.669900
30,2.532100
40,2.353300
50,2.110700
60,2.183000
70,2.174500
80,2.149000
90,2.156900
100,2.027700


TrainOutput(global_step=1000, training_loss=2.014699073791504, metrics={'train_runtime': 2332.5452, 'train_samples_per_second': 3.43, 'train_steps_per_second': 0.429, 'total_flos': 1.8181624017325056e+16, 'train_loss': 2.014699073791504, 'epoch': 0.08})

In [ ]:
DEVICE = "cuda:0"

In [ ]:
#Save trained model 
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.save_pretrained("trained-model")
model.push_to_hub("nebulae7/eight",use_auth_token=True,commit_description='initial')

adapter_model.bin:   0%|          | 0.00/75.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nebulae7/eight/commit/0afdc5e9df65b96b2f8d90be0872d7f45a5ece70', commit_message='Upload model', commit_description='', oid='0afdc5e9df65b96b2f8d90be0872d7f45a5ece70', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model.config.to_json_file("config.json")